In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import scanpy as sc
import igraph
import scvelo as scv
import loompy as lmp
import anndata
import os
import re
scv.set_figure_params(style="scvelo")
pl.rcParams["figure.figsize"] = (10,10)
Colorss=["#E41A1C","#377EB8","#4DAF4A","#984EA3","#FF7F00","#FFFF33","#A65628","#F781BF"]
#velocyto run10x /Volumes/Extreme\ Pro/D16_boost/ Users/fayzan/Downloads/refdata-gex-GRCh38-2020-A/genes/genes.gtf
%matplotlib inline


In [8]:
!pip list

Package                       Version
----------------------------- -----------
anndata                       0.9.2
anyio                         3.6.2
appnope                       0.1.3
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
asttokens                     2.2.1
attrs                         23.1.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
beautifulsoup4                4.12.2
bleach                        6.0.0
certifi                       2023.5.7
cffi                          1.15.1
charset-normalizer            3.1.0
click                         8.1.7
cloudpickle                   2.2.1
comm                          0.1.3
contourpy                     1.0.7
cycler                        0.11.0
debugpy                       1.6.7
decorator                     5.1.1
defusedxml                    0.7.1
entrypoints                   0.4
executing                     1.2.0
fastjsonschema                2.16.3
filelock    

In [4]:
!pip install scanpy
!pip install loompy
!pip install igraph
!pip install anndata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.1 MB/s eta 0:00:0000:0100:01


In [10]:
adata_WT1 = scv.read('/Users/fayzan/PycharmProjects/Betel/Aim2/SCVELO_Surv/GWT1.loom')
adata_KO1 = scv.read('/Users/fayzan/PycharmProjects/Betel/Aim2/SCVELO_Surv/GKO1.loom')
adata_WT2 = scv.read('/Users/fayzan/PycharmProjects/Betel/Aim2/SCVELO_Surv/GWT2.loom')
adata_KO2 = scv.read('/Users/fayzan/PycharmProjects/Betel/Aim2/SCVELO_Surv/GKO2.loom')


In [11]:
#Merge all velocyto files and then subset the cells. Then merge info.
adata_WT1.obs['newid'] = adata_WT1.obs.index
adata_WT2.obs['newid'] = adata_WT2.obs.index
adata_KO1.obs['newid'] = adata_KO1.obs.index
adata_KO2.obs['newid'] = adata_KO2.obs.index

In [12]:
adata_WT1.obs


,Clusters,_X,_Y,newid
CellID,,,,
GWT1:AAAGTGATCACCGACGx,1,-17.077444,-11.869936,GWT1:AAAGTGATCACCGACGx
GWT1:AACAAAGTCGGAGTGAx,9,12.098558,16.949923,GWT1:AACAAAGTCGGAGTGAx
GWT1:AAACGCTGTGTTCGATx,6,-4.913635,-31.655123,GWT1:AAACGCTGTGTTCGATx
GWT1:AAAGTCCCAACGACAGx,10,-13.780383,30.208845,GWT1:AAAGTCCCAACGACAGx
GWT1:AAAGTGAGTTCGAACTx,4,16.889030,-27.093758,GWT1:AAAGTGAGTTCGAACTx
...,...,...,...,...
GWT1:TTTGTTGGTCACTTAGx,0,-19.211367,18.296263,GWT1:TTTGTTGGTCACTTAGx
GWT1:TTTGTTGTCTAAGCCAx,8,37.145710,-3.072067,GWT1:TTTGTTGTCTAAGCCAx
GWT1:TTTGATCGTGCAATAAx,0,-37.280457,0.677654,GWT1:TTTGATCGTGCAATAAx


In [13]:
adata_WT1.obs['newid'] = adata_WT1.obs['newid'].str.replace(r'^.*?:', '',regex=True)
adata_WT2.obs['newid'] = adata_WT2.obs['newid'].str.replace(r'^.*?:', '',regex=True)
adata_KO1.obs['newid'] = adata_KO1.obs['newid'].str.replace(r'^.*?:', '',regex=True)
adata_KO2.obs['newid'] = adata_KO2.obs['newid'].str.replace(r'^.*?:', '',regex=True)

In [14]:
adata_WT1.obs['newid'] = adata_WT1.obs['newid'].str.replace(r'.$', '-1',regex=True)
adata_WT2.obs['newid'] = adata_WT2.obs['newid'].str.replace(r'.$', '-1',regex=True)
adata_KO1.obs['newid'] = adata_KO1.obs['newid'].str.replace(r'.$', '-1',regex=True)
adata_KO2.obs['newid'] = adata_KO2.obs['newid'].str.replace(r'.$', '-1',regex=True)
adata_WT1.obs

,Clusters,_X,_Y,newid
CellID,,,,
GWT1:AAAGTGATCACCGACGx,1,-17.077444,-11.869936,AAAGTGATCACCGACG-1
GWT1:AACAAAGTCGGAGTGAx,9,12.098558,16.949923,AACAAAGTCGGAGTGA-1
GWT1:AAACGCTGTGTTCGATx,6,-4.913635,-31.655123,AAACGCTGTGTTCGAT-1
GWT1:AAAGTCCCAACGACAGx,10,-13.780383,30.208845,AAAGTCCCAACGACAG-1
GWT1:AAAGTGAGTTCGAACTx,4,16.889030,-27.093758,AAAGTGAGTTCGAACT-1
...,...,...,...,...
GWT1:TTTGTTGGTCACTTAGx,0,-19.211367,18.296263,TTTGTTGGTCACTTAG-1
GWT1:TTTGTTGTCTAAGCCAx,8,37.145710,-3.072067,TTTGTTGTCTAAGCCA-1
GWT1:TTTGATCGTGCAATAAx,0,-37.280457,0.677654,TTTGATCGTGCAATAA-1


In [15]:
sample_obs = pd.read_csv("/Users/fayzan/PycharmProjects/Betel/Aim2/SCVELO_Surv/cellID_obsunique.csv")
sample_obs_all = pd.read_csv("/Users/fayzan/PycharmProjects/Betel/Aim2/SCVELO_Surv/cellID_obsnotunique.csv")
umap_cord = pd.read_csv("/Users/fayzan/PycharmProjects/Betel/Aim2/SCVELO_Surv/cell_embeddings.csv")
cell_clusters = pd.read_csv("/Users/fayzan/PycharmProjects/Betel/Aim2/SCVELO_Surv/clusters.csv")

In [16]:
sample_obs["x"]

0        AAACCCAAGCCATTTG-1
1        AAACCCAAGCGACCCT-1
2        AAACCCAAGCTACAAA-1
3        AAACCCAAGGAGGTTC-1
4        AAACCCAAGGTAAGTT-1
                ...        
43247    TTTGTTGGTTCAAGTC-1
43248    TTTGTTGTCCCAGCGA-1
43249    TTTGTTGTCGCGTTTC-1
43250    TTTGTTGTCTAGCCAA-1
43251    TTTGTTGTCTGTAACG-1
Name: x, Length: 43252, dtype: object

In [17]:
#Take Unique Cells
adata_WT1 = adata_WT1[np.isin(adata_WT1.obs.newid, sample_obs["x"])]
adata_KO1 = adata_KO1[np.isin(adata_KO1.obs.newid, sample_obs["x"])]
adata_WT2 = adata_WT2[np.isin(adata_WT2.obs.newid, sample_obs["x"])]
adata_KO2 = adata_KO2[np.isin(adata_KO2.obs.newid, sample_obs["x"])]


In [18]:
adata_WT1.var_names_make_unique()
adata_WT2.var_names_make_unique()
adata_KO1.var_names_make_unique()
adata_KO2.var_names_make_unique()

In [19]:
invitro_all = adata_WT1.concatenate(adata_KO2, adata_WT2,adata_KO1)

In [20]:
invitro_all.obs['newid1']=invitro_all.obs['newid']
invitro_all.obs['newid1'].loc[~invitro_all.obs['newid'].duplicated(keep='first')]=''

In [21]:
invitro_all.obs

,Clusters,_X,_Y,newid,batch,newid1
CellID,,,,,,
GWT1:AAAGTGATCACCGACGx-0,1,-17.077444,-11.869936,AAAGTGATCACCGACG-1,0,
GWT1:AACAAAGTCGGAGTGAx-0,9,12.098558,16.949923,AACAAAGTCGGAGTGA-1,0,
GWT1:AAACGCTGTGTTCGATx-0,6,-4.913635,-31.655123,AAACGCTGTGTTCGAT-1,0,
GWT1:AAAGTCCCAACGACAGx-0,10,-13.780383,30.208845,AAAGTCCCAACGACAG-1,0,
GWT1:AAAGTGAGTTCGAACTx-0,4,16.889030,-27.093758,AAAGTGAGTTCGAACT-1,0,
...,...,...,...,...,...,...
GKO1:TTTGTTGCAGCAGATGx-3,11,17.495062,40.408279,TTTGTTGCAGCAGATG-1,3,
GKO1:TTTGGAGAGGAAGTAGx-3,8,9.889330,26.435860,TTTGGAGAGGAAGTAG-1,3,
GKO1:TTTGACTGTCGTACTAx-3,1,-21.976715,-2.905352,TTTGACTGTCGTACTA-1,3,


In [22]:
invitro_all3 =  invitro_all[invitro_all.obs.newid1 == ""]

In [23]:
invitro_all3

View of AnnData object with n_obs × n_vars = 43244 × 36601
    obs: 'Clusters', '_X', '_Y', 'newid', 'batch', 'newid1'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'matrix', 'ambiguous', 'spliced', 'unspliced'

In [24]:
invitro_all3.obs

,Clusters,_X,_Y,newid,batch,newid1
CellID,,,,,,
GWT1:AAAGTGATCACCGACGx-0,1,-17.077444,-11.869936,AAAGTGATCACCGACG-1,0,
GWT1:AACAAAGTCGGAGTGAx-0,9,12.098558,16.949923,AACAAAGTCGGAGTGA-1,0,
GWT1:AAACGCTGTGTTCGATx-0,6,-4.913635,-31.655123,AAACGCTGTGTTCGAT-1,0,
GWT1:AAAGTCCCAACGACAGx-0,10,-13.780383,30.208845,AAAGTCCCAACGACAG-1,0,
GWT1:AAAGTGAGTTCGAACTx-0,4,16.889030,-27.093758,AAAGTGAGTTCGAACT-1,0,
...,...,...,...,...,...,...
GKO1:TTTGTTGCAGCAGATGx-3,11,17.495062,40.408279,TTTGTTGCAGCAGATG-1,3,
GKO1:TTTGGAGAGGAAGTAGx-3,8,9.889330,26.435860,TTTGGAGAGGAAGTAG-1,3,
GKO1:TTTGACTGTCGTACTAx-3,1,-21.976715,-2.905352,TTTGACTGTCGTACTA-1,3,


In [25]:
umap_cord['Unnamed: 0'] = sample_obs_all["x"]
umap_cord = umap_cord.drop_duplicates(['Unnamed: 0'])
umap_cord

,Unnamed: 0,UMAP_1,UMAP_2
0,AAACCCAAGCCATTTG-1,4.506885,-1.428377
1,AAACCCAAGCGACCCT-1,0.658285,-0.615243
2,AAACCCAAGCTACAAA-1,3.812819,-1.517525
3,AAACCCAAGGAGGTTC-1,1.731572,2.718117
4,AAACCCAAGGTAAGTT-1,-1.495209,0.777464
...,...,...,...
43431,TTTGTTGGTTCAAGTC-1,0.783719,-5.733409
43432,TTTGTTGTCCCAGCGA-1,1.029591,-3.637614
43433,TTTGTTGTCGCGTTTC-1,2.494064,-0.962188
43434,TTTGTTGTCTAGCCAA-1,-0.310459,-5.098460


In [26]:
cell_clusters['Unnamed: 0'] = sample_obs_all["x"]
cell_clusters = cell_clusters.drop_duplicates(['Unnamed: 0'])

In [27]:
combined = cell_clusters.merge(umap_cord, on="Unnamed: 0")

In [28]:
invitro_all3.obs['CellID1'] = invitro_all3.obs.index
invitro_all3.obs = invitro_all3.obs.merge(combined,left_on="newid", right_on="Unnamed: 0", how="left")

In [29]:
invitro_all3.obs["CellID"] = invitro_all3.obs["CellID1"]
invitro_all3.obs.index = invitro_all3.obs["CellID"]
invitro_all3.obs

,Clusters,_X,_Y,newid,batch,newid1,CellID1,Unnamed: 0,x,UMAP_1,UMAP_2,CellID
CellID,,,,,,,,,,,,
GWT1:AAAGTGATCACCGACGx-0,1,-17.077444,-11.869936,AAAGTGATCACCGACG-1,0,,GWT1:AAAGTGATCACCGACGx-0,AAAGTGATCACCGACG-1,1,-2.844779,-4.741085,GWT1:AAAGTGATCACCGACGx-0
GWT1:AACAAAGTCGGAGTGAx-0,9,12.098558,16.949923,AACAAAGTCGGAGTGA-1,0,,GWT1:AACAAAGTCGGAGTGAx-0,AACAAAGTCGGAGTGA-1,1,-3.589407,0.633650,GWT1:AACAAAGTCGGAGTGAx-0
GWT1:AAACGCTGTGTTCGATx-0,6,-4.913635,-31.655123,AAACGCTGTGTTCGAT-1,0,,GWT1:AAACGCTGTGTTCGATx-0,AAACGCTGTGTTCGAT-1,1,1.552553,1.454273,GWT1:AAACGCTGTGTTCGATx-0
GWT1:AAAGTCCCAACGACAGx-0,10,-13.780383,30.208845,AAAGTCCCAACGACAG-1,0,,GWT1:AAAGTCCCAACGACAGx-0,AAAGTCCCAACGACAG-1,1,-1.306827,-0.967202,GWT1:AAAGTCCCAACGACAGx-0
GWT1:AAAGTGAGTTCGAACTx-0,4,16.889030,-27.093758,AAAGTGAGTTCGAACT-1,0,,GWT1:AAAGTGAGTTCGAACTx-0,AAAGTGAGTTCGAACT-1,1,-1.542258,3.240822,GWT1:AAAGTGAGTTCGAACTx-0
...,...,...,...,...,...,...,...,...,...,...,...,...
GKO1:TTTGTTGCAGCAGATGx-3,11,17.495062,40.408279,TTTGTTGCAGCAGATG-1,3,,GKO1:TTTGTTGCAGCAGATGx-3,TTTGTTGCAGCAGATG-1,4,2.015658,-7.688165,GKO1:TTTGTTGCAGCAGATGx-3
GKO1:TTTGGAGAGGAAGTAGx-3,8,9.889330,26.435860,TTTGGAGAGGAAGTAG-1,3,,GKO1:TTTGGAGAGGAAGTAGx-3,TTTGGAGAGGAAGTAG-1,4,-0.598763,-2.847061,GKO1:TTTGGAGAGGAAGTAGx-3
GKO1:TTTGACTGTCGTACTAx-3,1,-21.976715,-2.905352,TTTGACTGTCGTACTA-1,3,,GKO1:TTTGACTGTCGTACTAx-3,TTTGACTGTCGTACTA-1,1,2.514450,-1.494023,GKO1:TTTGACTGTCGTACTAx-3


In [30]:
X_umap = np.stack([invitro_all3.obs["UMAP_1"], invitro_all3.obs["UMAP_2"]]).T

In [31]:
invitro_all3.obsm['X_umap'] = X_umap


In [32]:
invitro2 =invitro_all3

In [33]:
scv.pp.filter_and_normalize(invitro2, min_shared_counts=30, n_top_genes=4000)

Filtered out 25057 genes that are detected 30 counts (shared).
Normalized count data: X, spliced, unspliced.
Extracted 4000 highly variable genes.


/Users/fayzan/opt/anaconda3/envs/PGS/lib/python3.8/site-packages/scvelo/preprocessing/utils.py:705: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  log1p(adata)


Logarithmized X.


In [ ]:
scv.pp.moments(invitro2, n_pcs=100, n_neighbors=20)
scv.tl.recover_dynamics(invitro2)
scv.tl.velocity(invitro2, mode='dynamical')
scv.tl.velocity_graph(invitro2)
scv.tl.recover_latent_time(invitro2, t_max=20000)

In [38]:
scv.pl.velocity_embedding_stream(invitro2,color='x', title='Survival_SCVELO')

In [46]:
scv.pl.velocity(invitro2, ['TP53'], ncols=2)

In [61]:
%matplotlib inline
plt.show()
%matplotlib notebook

In [62]:
scv.tl.rank_velocity_genes(invitro2, groupby='x', min_corr=.15)

df = scv.DataFrame(invitro2.uns['rank_velocity_genes']['names'])
df.head(100)


In [60]:
!pip install tornado --upgrade

In [63]:
scv.pl.heatmap(invitro2, var_names=["ALDH1A1", "LMO3", "KCNJ6", "AGTR1", "EN1", "TH", "NR4A2", "FOXA1","SOX6", "SOX5", "VGF", "CALB1", "CALB2", "OTX2", "BAX", "JUN"], sortby='latent_time', col_color='x', n_convolve=100, colorbar=True, sort=True)

In [65]:
scv.tl.velocity_pseudotime(invitro2)
scv.pl.scatter(invitro2, color='latent_time', cmap='gnuplot')
